In [ ]:
# stt_llama_chat_safe.py

import argparse
import os
import time
from collections import deque
import numpy as np
import sounddevice as sd
import soundfile as sf
import webrtcvad
import whisper
import tempfile

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# ------------------------------
# 1️⃣ VAD 기반 녹음 함수
# ------------------------------
def record_with_vad(max_seconds=90, fs=16000, frame_ms=20, vad_mode=1, silence_sec=3):
    vad = webrtcvad.Vad()
    vad.set_mode(vad_mode)

    frame_len = int(fs * frame_ms / 1000)
    silence_win = int(silence_sec * 1000 / frame_ms)
    recent = deque(maxlen=max(silence_win, 1))

    print("🎤 녹음 시작 (말씀하세요). 무음이 되면 자동 종료됩니다...")

    recorded = []
    started = time.time()
    speech_started = False

    def callback(indata, frames, time_info, status):
        recorded.append(indata.copy())

    try:
        with sd.InputStream(samplerate=fs, channels=1, dtype='int16', callback=callback):
            while True:
                time.sleep(frame_ms / 1000.0)
                if not recorded:
                    continue

                frame = recorded[-1].flatten()
                if len(frame) >= frame_len:
                    speech = vad.is_speech(frame[:frame_len].tobytes(), sample_rate=fs)
                    recent.append(1 if speech else 0)
                    if speech:
                        speech_started = True

                if speech_started and len(recent) == recent.maxlen and sum(recent) == 0:
                    print("🛑 무음 지속 → 종료")
                    break

                if time.time() - started > max_seconds:
                    print("⏱ 최대 녹음 시간 도달, 종료합니다.")
                    break

    except KeyboardInterrupt:
        print("\n사용자 중단")

    if not recorded:
        return None, fs

    audio = np.concatenate(recorded, axis=0).astype(np.int16)
    return audio, fs


# ------------------------------
# 2️⃣ Whisper STT 함수
# ------------------------------
def run_stt(audio, fs, model_name="base", task="transcribe", device="cpu"):
    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    tmp_path = tmp.name
    tmp.close()
    sf.write(tmp_path, audio, fs)
    print(f"💾 임시 파일 저장: {tmp_path}")

    print("🌀 Whisper 모델 로드 중...")
    model = whisper.load_model(model_name, device=device)

    print(f"📖 변환 중... (task={task})")
    result = model.transcribe(tmp_path, task=task)
    text = result.get("text", "").strip()

    try:
        os.remove(tmp_path)
    except Exception:
        pass

    return text


# ------------------------------
# 3️⃣ LLaMA 대화 + 스트리밍
# ------------------------------
def run_llama_conversation(hf_token, difficulty, duration=600, device="cpu"):
    print("🤖 LLaMA 모델 로드 중...")
    model_name = "meta-llama/Llama-3.2-1B"

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token).to(device)

    history = []
    start_time = time.time()

    while True:
        elapsed = time.time() - start_time
        is_final = elapsed > duration  # 종료 직전 플래그

        # 🎤 사용자 발화
        audio, fs = record_with_vad(max_seconds=20, fs=16000, vad_mode=3, silence_sec=2)
        if audio is None:
            print("❌ 녹음 실패, 대화 종료")
            break

        stt_text = run_stt(audio, fs, model_name="base", task="transcribe", device="cpu")
        if not stt_text:
            print("❌ 음성 인식 결과 없음")
            continue

        history.append(("사용자", stt_text))

        # 📖 프롬프트 구성
        dialogue = "\n".join([f"{role}: {msg}" for role, msg in history])

        if is_final:
            prompt = f"""
너는 {difficulty} 답변을 해주는 조교야.
지금까지 사용자의 대화를 정리하고 마지막으로 따뜻하게 마무리 인사를 해줘.
예: '오늘 대화 수고했어! 다음에 또 이야기하자.'

{dialogue}
AI:"""
        else:
            prompt = f"""
너는 {difficulty} 답변을 해주는 조교야.
아래는 사용자와 AI의 대화다. 반드시 {difficulty} 답변을 해라.

{dialogue}
AI:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        print("\n===== 🤖 AI 답변 (Streaming) =====")
        streamer = TextStreamer(tokenizer)
        output = model.generate(**inputs, max_new_tokens=256, streamer=streamer)

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        ai_answer = response.split("AI:")[-1].strip()
        history.append(("AI", ai_answer))
        print("\n===============================\n")

        if is_final:
            print("⏰ 대화 10분 종료, 프로그램을 마무리합니다.")
            break


# ------------------------------
# 4️⃣ 메인
# ------------------------------
def main():
    parser = argparse.ArgumentParser(description="STT + LLaMA Chatbot (대화 히스토리 + 난이도 유지)")
    parser.add_argument("--difficulty", default="쉽게", help="답변 난이도 (예: 쉽게, 어렵게, 자세히)")
    parser.add_argument("--duration", type=int, default=600, help="대화 지속 시간(초, 기본 600초=10분)")
    args = parser.parse_args()

    hf_token = os.getenv("HF_TOKEN")  # ✅ 환경 변수에서 안전하게 가져오기
    if not hf_token:
        print("❌ 환경 변수 HF_TOKEN에 Hugging Face 토큰을 설정하세요.")
        return

    run_llama_conversation(hf_token, args.difficulty, duration=args.duration, device="cpu")


if __name__ == "__main__":
    main()
